## Summary

**New**: Pipeline details

How does RAG(retrieval-augmented generation) work?
        


(1.31)New model: [Bard(Original)](https://bard.google.com/chat) For 13 questions from paper, [13 answers](https://g.co/bard/share/117570e457af) For 7 new questions, [7 answers](https://g.co/bard/share/644ebd6a4f50) API [Join Waitlist](https://cloud.google.com/ai/earlyaccess/join?hl=en)

(1.22) View the all [answers](yl_Appendix.ipynb).

(1.16) 7 questions:

1. What is the best estimate of the equilibrium climate sensitivity?
2. Is it possible the Arctic will become ice free at some points before 2050?
3. How likely will the world see runaway ice loss from the Antarctic  ice sheets by 2100?
4. To what extent has climate change impacted ice sheets in both polar regions?
5. Will coastal New York see the impact of Greenland ice sheet melting by 2100?
6. What will be the likely changes to weather and climate extremes in coastal New York around 2050 if the world take no action to reduce greenhouse gas emissions?
7. What are the primary evidence and how robust is it that human activities caused the observed global warming since the pre-industrial period?

The 13 original questions are from [chatClimate: Grounding Conversational AI in Climate Science](https://arxiv.org/abs/2304.05510)



The Climate report is from [IPCC_AR6_WGIII_FullReport](https://sdgs.un.org/sites/default/files/2023-01/IPCC_AR6_WGIII_FullReport.pdf)

(1.2)The Llama2(Original), ChatGPT-4(Original), Llama2+(Climate report) and Llama2 hybird code parts are available [here](https://github.com/yl1127/Academic-projects/blob/main/yl_Climate_LLM/yl_Climate_0102.ipynb).


(2023.12.18)The Llama2(Original) and Llama2+(Climate report) code parts are available [here](https://github.com/yl1127/Academic-projects/blob/main/yl_Climate_LLM/yl_LlamaCloud_Climate.ipynb).


<!-- | Rate | Llama2(Original) | GPT-4(Original) | Llama2 +(Climate report)|Llama2 hybird|
| ----------- | ----------- |------------------|-------------------------|-----------------------|
|Q1: What is the best estimate of the equilibrium climate sensitivity?| -- | -- | -- |  --|
|Q2: Is it possible the Arctic will become ice free at some points before 2050?| -- | -- | -- |  --|
|Q3: How likely will the world see runaway ice loss from the Antarctic  ice sheets by 2100?| -- | -- | -- |  --|
|Q4: To what extent has climate change impacted ice sheets in both polar regions?| -- | -- | -- |  --|
|Q5: Will coastal New York see the impact of Greenland ice sheet melting by 2100?| -- | -- | -- |  --|
|Q6: What will be the likely changes to weather and climate extremes in coastal New York around 2050 if the world take no action to reduce greenhouse gas emissions?| -- | -- | -- |  --|
|Q7: What are the primary evidence and how robust is it that human activities caused the observed global warming since the pre-industrial period?| -- | -- | -- |  --| -->

## This demo app shows:
* How to run Llama2 in the cloud hosted on Replicate
* How to use LangChain to ask Llama general questions and follow up questions
* How to use LangChain to load a recent PDF doc - This is the well known RAG (Retrieval Augmented Generation) method to let LLM such as Llama2 be able to answer questions about the data not publicly available when Llama2 was trained, or about your own data. RAG is one way to prevent LLM's hallucination
* How to use LangChain to load a directory which includes multiple PDF docs.
* You should also review the [HelloLlamaLocal](HelloLlamaLocal.ipynb) notebook for more information on RAG

**Note** We will be using Replicate to run the examples here. You will need to first sign in with Replicate with your github account, then create a free API token [here](https://replicate.com/account/api-tokens) that you can use for a while. 
After the free trial ends, you will need to enter billing info to continue to use Llama2 hosted on Replicate.

Let's start by installing the necessary packages:
- sentence-transformers for text embeddings
- chromadb gives us database capabilities 
- langchain provides necessary RAG tools for this demo

And setting up the Replicate token.

### How to run original Llama2 and ChatGPT?

In [1]:
# !pip install langchain replicate sentence-transformers chromadb pypdf

In [3]:
from getpass import getpass
import os

Next we call the ChatGPT model from OpenAI. In this example we will use the ChatGPT-3.5-turbo chat model. You can find more ChatGPT models by searching for them on the [OpenAI API keys](https://platform.openai.com/api-keys).


In [4]:
# OpenAI API
import os
import openai
from openai import OpenAI

# from dotenv import load_dotenv, find_dotenv
# _ = load_dotenv(find_dotenv()) # read local .env file
new_api_key = getpass()

os.environ["OPENAI_API_KEY"] = new_api_key
openai.api_key = os.environ['OPENAI_API_KEY']

openai_client = OpenAI()

 ········


With the model set up, you are now ready to ask some questions. Here is an example of the simplest way to ask the model some general questions.

### How to load an external PDF and do embedding?

In [5]:
# !pip install pypdf

In [6]:
## Load the Report and show the first page content
from pypdf import PdfReader
loader = PdfReader("IPCC_AR6_WGIII_FullReport.pdf")
pdf_texts = [p.extract_text().strip() for p in loader.pages] # remove some empty contents
pdf_texts = [text for text in pdf_texts if text]
print(pdf_texts[0])

Working Group III contribution to the
Sixth Assessment Report of the 
Intergovernmental Panel on Climate ChangeWGIIIMitigation of Climate ChangeClimate Change 2022


In [15]:
## check docs length and content
print(len(pdf_texts))
print(pdf_texts[3]) # any pages between 0 to 2030(len(pdf_texts))

2030
Foreword  
and Preface


We need to store our documents. There are more than 30 vector stores (DBs) supported by LangChain. 
For this example we will use [Chroma](https://python.langchain.com/docs/integrations/vectorstores/chroma) which is light-weight and in memory so it's easy to get started with.
For other vector stores especially if you need to store a large amount of data - see https://python.langchain.com/docs/integrations/vectorstores

In [16]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, SentenceTransformersTokenTextSplitter


In [34]:
## Split the report as small pieces(chunks)
character_splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", ". ", " ", ""],
    chunk_size=1000,
    chunk_overlap=20
) # Each pieces has 1000 unit length
character_split_texts = character_splitter.split_text('\n\n'.join(pdf_texts))

print(character_split_texts[10])
print(f"\nTotal chunks: {len(character_split_texts)}") # Total number of small chunks(13302)

approaches falling outside the scope of individual sectors. 
As in the AR5, there is a chapter on recent trends and drivers, with 
the scope expanded to cover historic emissions and recent policy 
developments. Following the pattern established in the WG III AR5 
report, and the Special Report on Global Warming of 1.5°C, this 
report assesses published emission scenarios with a 21st century 
perspective. Modelled emission scenarios are categorised according 
to climate outcomes, allowing a handshake with the WG I assessment. 
To meet the goal of linking top-down and bottom-up insights, the 
report includes an additional pathways chapter that provides a mid-
century perspective, focussing on national and regional scales and the 
alignment between development pathways and mitigation actions.
As in the AR5, this report addresses mitigation enablers such as 
international cooperation, finance and investment, and policies and

Total chunks: 13302


To store the documents, we will need to split them into chunks using [`RecursiveCharacterTextSplitter`](https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter) and create vector representations of these chunks using [`HuggingFaceEmbeddings`](https://www.google.com/search?q=langchain+hugging+face+embeddings&sca_esv=572890011&ei=ARUoZaH4LuumptQP48ah2Ac&oq=langchian+hugg&gs_lp=Egxnd3Mtd2l6LXNlcnAiDmxhbmdjaGlhbiBodWdnKgIIADIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCjIHEAAYgAQYCkjeHlC5Cli5D3ABeAGQAQCYAV6gAb4CqgEBNLgBAcgBAPgBAcICChAAGEcY1gQYsAPiAwQYACBBiAYBkAYI&sclient=gws-wiz-serp) on them before storing them into our vector database. 

In general, you should use larger chuck sizes for highly structured text such as code and smaller size for less structured text. You may need to experiment with different chunk sizes and overlap values to find out the best numbers.

In [20]:
## Embedding the words to vectors
import chromadb
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction

embedding_function = SentenceTransformerEmbeddingFunction()
print(embedding_function([character_split_texts[10]])) # An example of embedding result

[[-0.0008131897775456309, 0.005816067568957806, 0.07490230351686478, 0.06837400794029236, 0.08766964823007584, -0.0012497920542955399, -0.0640428364276886, -0.041942812502384186, -0.07316127419471741, 0.01825740560889244, -0.08234508335590363, -0.05028411000967026, -0.03627554699778557, 0.057594843208789825, -0.026721015572547913, 0.020879218354821205, -0.11043477058410645, -0.0446099191904068, -0.06738457083702087, -0.09205488860607147, 0.025163700804114342, 0.053215134888887405, 0.034258224070072174, -0.010387456975877285, 0.018884461373090744, -0.05462275445461273, -0.025599760934710503, 0.023365968838334084, -0.06750442087650299, 0.08059892803430557, 0.0327172689139843, 0.08507392555475235, -0.0669960305094719, -0.016989467665553093, 0.02977531962096691, 0.0656718760728836, 0.03229948505759239, 0.01954924501478672, 0.03471197932958603, 0.004731947090476751, 0.034873683005571365, -0.08469009399414062, 0.0052584330551326275, -0.05337139219045639, -0.03895178809762001, 0.0231067351996

In [ ]:
## Add all embedding results to Database(Long term memory) and indexed them

chroma_client = chromadb.Client()
chroma_collection = chroma_client.create_collection("IPCC_AR6_WGIII_FullReport.pdf", embedding_function=embedding_function)

ids = [str(i) for i in range(len(character_split_texts))]

chroma_collection.add(ids=ids, documents=character_split_texts)
chroma_collection.count() # indexed embedding chunks in memory(should be the same as number of chunks)

In [36]:
# Embedding our question and do the semantic similarity search
# print out the top 5 similar chunks

# question = "Is it still possible to limit warming to 1.5°C?" #Q1
query = "Is it still possible to limit warming to 1.5°C?" #Q1

results = chroma_collection.query(query_texts=[query], n_results=5)
retrieved_documents = results['documents'][0]

for document in retrieved_documents:
    print(document)
    print('\n')

agricultural markets and food security (Havlík et al. 2014; Hasegawa 
et al. 2018; Doelman et al. 2019; Fujimori et al. 2019). Mitigation 
policies aimed at achieving  1.5°C–2°C, if not managed properly, Box 3.6 (continued)
20
051015tCO2-eq yr–1
2030 2050
YearC1: limit warming 
to 1.5°C (>50%) 
with no or limited 
overshoot
C2: return 
warming to 1.5°C 
(>50%) after a 
high overshoot
C3: limit warming 
to 2°C (>67%)
C4: limit warming 
to 2°C (>50%)
C5: limit warming 
to 2.5°C (>50%)
C6: limit warming 
to 3°C (>50%)
C7: limit warming 
to 4°C (>50%)
C8: exceed 
warming of 4°C 
(≥50%)
Box 3.6, Figure 1 | Difference in per-capita emissions of Kyoto gases between the highest emitting and the lowest emitting of the 10 regions, 
in 2030 and 2050, by temperature category of pathways.
Through avoiding impacts of climate change, which fall more heavily on low-income countries, communities and households, and 
exacerbate poverty, mitigation reduces inequalities and poverty (Section 3.6.4.2).


of

## ChatGPT

Our external knowledge is ready. Next is about ChatGPT.

### How to run ChatGPT-3.5 + pdf (chatClimate)?

In [22]:
from langchain.chains import RetrievalQA
import langchain
langchain.debug = True

def get_chatclimate(query, retrieved_documents, model="gpt-3.5-turbo"):
    information = "\n\n".join(retrieved_documents)

    messages = [
        {
            "role": "system",
            "content": "You are a Q&A bot, an intelligent system that answers user questions ONLY based on the information provided by the user. If you use user information, please indicate the Page and Reference, which are provided below each piece of information. If the information cannot be found in the information provided by the user, please say ’I don’t know’"
        },
        {"role": "user", "content": f"Question: {query}. \n Information: {information}"}
    ]
    
    response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
    )
    content = response.choices[0].message.content
    return content


In [23]:
# question = "Is it still possible to limit warming to 1.5°C?" #Q1
query = "Is it still possible to limit warming to 1.5°C?" #Q1

results = chroma_collection.query(query_texts=[query], n_results=5)
retrieved_documents = results['documents'][0]

# for document in retrieved_documents:
#     print(document)
#     print('\n')

output = get_chatclimate(query=query, retrieved_documents=retrieved_documents)

print(output)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Based on the information provided, it is still possible to limit warming to 1.5°C. Modelled scenarios in Category C1 aim to limit warming to 1.5°C with a likelihood greater than 50% and reach or exceed warming of 1.5°C during the 21st century with a likelihood of 67% or less. These scenarios are referred to as scenarios that limit warming to 1.5°C (>50%) with no or limited overshoot. Limited overshoot refers to exceeding 1.5°C global warming by up to about 0.1°C and for up to several decades (Box 3.6). Therefore, these modelled scenarios show that it is possible to limit warming to 1.5°C, but there are potential challenges and obstacles to overcome (Table TS.1).


### How to run ChatGPT-3.5 hybrid chatClimate

In [25]:
question = "Is it still possible to limit warming to 1.5°C?" #Q1

def get_hybrid(query, retrieved_documents, model="gpt-3.5-turbo"):
    information = "\n\n".join(retrieved_documents)

    messages = [
        {
            "role": "system",
            "content": "You are a Q&A bot, an intelligent system that answers user questions based on the information provided by the user above the question and your in-house knowledge. There are 5 pieces of extra information above the user question. please indicate the Page and Reference, which are provided below each piece of information. Additionally, let us know which part of your answer is from the IPCC information and which part is based on your in-house knowledge by writing either (IPCC AR6) or (Inhouse knowledge). If the information cannot be found in the information provided by the user or your in-house knowledge, please say ’I don’t know’."
        },
        {"role": "user", "content": f"Question: {query}. \n Information: {information}"}
    ]
    
    response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
    )
    content = response.choices[0].message.content
    return content

In [26]:
# question = "Is it still possible to limit warming to 1.5°C?" #Q1
query = "Is it still possible to limit warming to 1.5°C?" #Q1

# query = joint_query

results = chroma_collection.query(query_texts=[query], n_results=5)
retrieved_documents = results['documents'][0]

# for document in retrieved_documents:
#     print(document)
#     print('\n')

output = get_hybrid(query=query, retrieved_documents=retrieved_documents)

print(output)

According to the information provided, there are modelled scenarios (Category C1) that limit warming to 1.5°C in 2100 with a likelihood of greater than 50%. These scenarios are referred to as scenarios that limit warming to 1.5°C (>50%) with no or limited overshoot. Limited overshoot refers to exceeding 1.5°C global warming by up to about 0.1°C and for up to several decades (IPCC AR6). However, it is important to note that there are still many obstacles to overcome in achieving these scenarios (IPCC AR6).


### How does ChatGPT-4 answer?

In [27]:
def get_ChatGPT(query, model="gpt-4"):
    # information = "\n\n".join(retrieved_documents)

    messages = [
        {
            "role": "system",
            "content": "You are Q&A bot. A highly intelligent system that answers user questions"
        },
        {"role": "user", "content": f"Question: {query}."}
    ]
    
    response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
    )
    content = response.choices[0].message.content
    return content

In [28]:
query = "Is it still possible to limit warming to 1.5°C?" #Q1
output = get_ChatGPT(query=query)

print(output)

Answer: Studies suggest that it is technically possible to limit global warming to 1.5°C, but it would require substantial and immediate reductions in greenhouse gas emissions. This includes rapid, far-reaching changes in all aspects of society, like energy, land use, cities, and industry. However, achieving this goal remains highly challenging due to a variety of political, economic, and technological obstacles.


### How to run ChatGPT-4+pdf(chatClimate)?

In [29]:
question = "Is it still possible to limit warming to 1.5°C?" #Q1

from langchain.chains import RetrievalQA

def get_chatclimate(query, retrieved_documents, model="gpt-4"):
    information = "\n\n".join(retrieved_documents)

    messages = [
        {
            "role": "system",
            "content": "You are a Q&A bot, an intelligent system that answers user questions ONLY based on the information provided by the user. If you use user information, please indicate the Page and Reference, which are provided below each piece of information. If the information cannot be found in the information provided by the user, please say ’I don’t know’"
        },
        {"role": "user", "content": f"Question: {query}. \n Information: {information}"}
    ]
    
    response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
    )
    content = response.choices[0].message.content
    return content
    

In [30]:
# question = "Is it still possible to limit warming to 1.5°C?" #Q1
query = "Is it still possible to limit warming to 1.5°C?" #Q1

results = chroma_collection.query(query_texts=[query], n_results=5)
retrieved_documents = results['documents'][0]

# for document in retrieved_documents:
#     print(document)
#     print('\n')

output = get_chatclimate(query=query, retrieved_documents=retrieved_documents)

print(output)

Based on the provided information, it appears that it is theoretically possible to limit warming to 1.5°C. There are modelled scenarios, such as Category C1 and C2, that aim to limit warming to this level.  Category C1 is about limiting warming to 1.5°C in 2100 with a likelihood of greater than 50%, with no or limited overshoot. Limited overshoot refers to momentarily exceeding 1.5°C global warming by up to about 0.1°C for up to several decades. Category C2 represents scenarios that return warming to 1.5°C in 2100 with a likelihood of greater than 50% after a high overshoot. High overshoot refers to exceeding 1.5°C global warming by 0.1°C–0.3°C for up to several decades. However, the provided information mentions that there are many obstacles yet to overcome to achieve these temperature targets (Page 1, third to last paragraph).


### How to run ChatGPT-4 hybird chatClimate?

In [31]:
question = "Is it still possible to limit warming to 1.5°C?" #Q1

from langchain.chains import RetrievalQA

def get_hybrid(query, retrieved_documents, model="gpt-4"):
    information = "\n\n".join(retrieved_documents)

    messages = [
        {
            "role": "system",
            "content": "You are a Q&A bot, an intelligent system that answers user questions based on the information provided by the user above the question and your in-house knowledge. There are 5 pieces of extra information above the user question. please indicate the Page and Reference, which are provided below each piece of information. Additionally, let us know which part of your answer is from the IPCC information and which part is based on your in-house knowledge by writing either (IPCC AR6) or (Inhouse knowledge). If the information cannot be found in the information provided by the user or your in-house knowledge, please say ’I don’t know’."
        },
        {"role": "user", "content": f"Question: {query}. \n Information: {information}"}
    ]
    
    response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
    )
    content = response.choices[0].message.content
    return content



In [32]:
# question = "Is it still possible to limit warming to 1.5°C?" #Q1
query = "Is it still possible to limit warming to 1.5°C?" #Q1

results = chroma_collection.query(query_texts=[query], n_results=5)
retrieved_documents = results['documents'][0]

# for document in retrieved_documents:
#     print(document)
#     print('\n')

output = get_hybrid(query=query, retrieved_documents=retrieved_documents)

print(output)

Yes, it is still theoretically possible to limit warming to 1.5°C. According to the information provided, there are modelled scenarios that predict limiting warming to 1.5°C in 2100 with a likelihood of greater than 50% and scenarios that return warming to 1.5°C (>50%) after a high overshoot. The high overshoot refers to temporarily exceeding 1.5°C global warming by 0.1°C–0.3°C for up to several decades (Page: Box 3.6, Figure 1, Reference 48). My answer is based on the information from IPCC AR6. However, it is important to add (Inhouse knowledge) that reaching these goals would require significant efforts to reduce greenhouse gas emissions immediately as well as implement various mitigation strategies.
